# GNAO1 DEE17
to do short intro to DEE17

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.56


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [4]:
df = pd.read_excel("input/GNAO1_DEE17_individuals.xlsx")
df.head(20)

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,age_of_onset,age_at_last_encounter,sex,HPO,Tonic seizure,EEG with burst suppression,Hypsarrhythmia,Dystonia,Chorea,Athetosis
0,str,str,str,optional str,str,str,str,str,str,optional str,age,age,M:F:O:U,na,HP:0032792,HP:0010851,HP:0002521,HP:0001332,HP:0002072,HP:0002305
1,PMID:23993195,"De Novo mutations in GNAO1, encoding a Gαo subunit of heterotrimeric G proteins, cause epileptic encephalopathy",Individual 1,NaN,OMIM:615473,Developmental and epileptic encephalopathy 17,NM_020988.3,c.836T>A,na,RCV000056405.29,P4D,P13Y,F,na,P4D,P4D,excluded,excluded,excluded,excluded
2,PMID:23993195,"De Novo mutations in GNAO1, encoding a Gαo subunit of heterotrimeric G proteins, cause epileptic encephalopathy",Individual 2,NaN,OMIM:615473,Developmental and epileptic encephalopathy 17,NM_020988.3,c.521A>G,na,VCV000066113.3,P29D,P4Y1M,F,na,P4Y1M,P4Y1M,P3M,excluded,excluded,excluded


In [5]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 20 fields


In [6]:
GNAO1_transcript = "NM_020988.3"
vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="GNAO1",
                          transcript=GNAO1_transcript,
                          allele_1_column_name="allele_1")

[INFO] encoding variant "c.521A>G"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_020988.3%3Ac.521A>G/NM_020988.3?content-type=application%2Fjson
[INFO] encoding variant "c.836T>A"
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_020988.3%3Ac.836T>A/NM_020988.3?content-type=application%2Fjson


In [7]:
vmanager.to_summary()

,status,count,alleles
0,mapped,2,"c.521A>G, c.836T>A"
1,unmapped,0,


In [8]:
vmanager.add_variants_to_individuals(individuals)

In [10]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1,
                                allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [11]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Individual 1 (FEMALE; P13Y),Developmental and epileptic encephalopathy 17 (OMIM:615473),NM_020988.3:c.836T>A (heterozygous),P4D: Tonic seizure (HP:0032792); EEG with burst suppression (HP:0010851)excluded: Hypsarrhythmia (HP:0002521); excluded: Dystonia (HP:0001332); excluded: Chorea (HP:0002072); excluded: Athetosis (HP:0002305)
Individual 2 (FEMALE; P4Y1M),Developmental and epileptic encephalopathy 17 (OMIM:615473),NM_020988.3:c.521A>G (heterozygous),P3M: Hypsarrhythmia (HP:0002521)P4Y1M: Tonic seizure (HP:0032792); EEG with burst suppression (HP:0010851)excluded: Dystonia (HP:0001332); excluded: Chorea (HP:0002072); excluded: Athetosis (HP:0002305)


In [12]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 2 GA4GH phenopackets to the directory phenopackets
